# Welcome to Hanasu

In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/wavs.zip', 'r') as zip_ref: zip_ref.extractall('zipfolder')

In [ ]:
%cd /content/drive/MyDrive/hanasu/hanasu
!pip install -r requirements.txt
%cd /content/drive/MyDrive/vits2_pytorch-main/monotonic_align
!python setup.py build_ext --inplace
%cd ../
!apt-get update && apt-get install -y espeak-ng
!pip install phonemizer

# Preprocess the dataset

In [ ]:
from text import preprocess_filelists
preprocess_filelists(["configs/Yuna/transcript-huge.txt"])

# Fix Bugs

In [ ]:
!pip install librosa==0.9.1
!pip install tensorboard==2.12.0 tensorflow==2.12.0
!pip install matplotlib==3.7.0 # This fixes: 'FigureCanvasAgg' object has no attribute 'tostring_rgb'
!pip install numpy==1.26.4

In [ ]:
import os
directory = '/content/Yuna/wavs'
for filename in os.listdir(directory):
  if filename.endswith(".mel.pt"):
    os.remove(os.path.join(directory, filename))

# Training

In [ ]:
!bash train.sh /content/Yuna/config.json 1

# Inference

In [ ]:
%cd ../hanasu

In [ ]:
from models import inference
from scipy.io.wavfile import write
import sounddevice as sd

In [ ]:
text ="A text view always uses exactly the amount of space it needs to display its rendered contents, but you can affect the view’s layout. For example, you can use the frame(width:height:alignment:) modifier to propose specific dimensions to the view. If the view accepts the proposal but the text doesn’t fit into the available space, the view uses a combination of wrapping, tightening, scaling, and truncation to make it fit. With a width of 100 points but no constraint on the height, a text view might wrap a long string."
# Or you can read from a file:
# with open("text.txt", 'r', encoding='utf-8') as f: text = f.read()

In [ ]:
# No streaming inference
result = inference(
    model_path="/Users/yuki/Downloads/G_158000.pth",
    text=text,
    config_path="./configs/config.json",
    noise_scale=0.2,
    noise_scale_w=1.0,
    length_scale=1.0,
    device="mps",
    stream=False,
)

write(data=result, rate=48000, filename="sample_vits2.wav")

In [ ]:
# Streaming inference
audio_generator = inference(
    model_path="/Users/yuki/Downloads/G_158000.pth",
    text=text,
    config_path="./configs/config.json",
    noise_scale=0.2,
    noise_scale_w=1.0,
    length_scale=1.0,
    device="mps",
    stream=True,
)

for audio_chunk in audio_generator:
    # Play each chunk immediately as it's generated
    sd.play(audio_chunk, samplerate=48000)
    sd.wait()  # Wait for chunk to finish playing
    print(f"Played chunk of {len(audio_chunk)} samples")